In [1]:
import openai
from utils import ChatSession,update_investor_profile
import pandas as pd

# load and set our key
openai.api_key = open("key.txt", "r").read().strip("\n")

# questions:
#         'Is the Customer older than 50? Say yes or no.',\
#         'Does the Customer earn less than 100K annually? Say yes or no.',\
#         'Does the Customer have a high risk appetite? Say yes or no.'

RuleBasedPortfolios = pd.read_excel('RuleBasedPortfolios.xlsx')
RuleBasedPortfolios.columns = RuleBasedPortfolios.columns.map(lambda x: x.lower())

RuleBasedPortfolios.age = RuleBasedPortfolios.age.apply(lambda x: 'yes' if x == '50 -' else 'no') # use .strip()
RuleBasedPortfolios.income = RuleBasedPortfolios.income.apply(lambda x: 'yes' if x == '0 - 100' else 'no') # use .strip()
RuleBasedPortfolios['risk appetite'] = RuleBasedPortfolios['risk appetite'].apply(lambda x: 'yes' if x == 'High' else 'no') # use .strip()

# GPT/GPT

In [2]:
session1 = ChatSession(gpt_name='Advisor')
session2 = ChatSession(gpt_name='Customer')

beginning_sentence = "Hi. I am not sure about how to invest. Could you help me?"
# beginning_sentence = "Hi. I am curious about today's investment chances. Could you help me?"

session2.inject(line="You are a customer at a bank, seeking financial advise from me. You will ask general questions about current financial situation in the world and Switzerland. You are 50 years old with an income of 120K. Your risk appetite is low. Do not give these information unless you are asked for. Say ok if you understand.",role="user")
session2.inject(line="Ok.",role= "assistant")
session2.inject(line=beginning_sentence,role= "assistant")

print('Customer: ' + beginning_sentence)

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

investor_profile = {i:False for i in ['age','income','risk appetite']}
while True:
    session1.chat(user_input=session2.messages[-1],temperature=0.01)
    session2.chat(user_input=session1.messages[-1])
    update_investor_profile(investor_profile=investor_profile,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'{session2.gpt_name}: {session2.messages[-1].content}')

    
    if not len([i for i in investor_profile.values() if not i]):
        assert sum(map(lambda x: 'no' in x.lower() or 'yes' in x.lower(),investor_profile.values())) == len(investor_profile.keys()), investor_profile
        break

# Rule based portfolio by using ``investor_profile``
portfolio = RuleBasedPortfolios.where(lambda x: x['age'].apply(lambda y: y in investor_profile['age'].lower())*\
                            x['income'].apply(lambda y: y in investor_profile['income'].lower())*\
                                x['risk appetite'].apply(lambda y: y in investor_profile['risk appetite'].lower()))['portfolio'].dropna().values
assert portfolio.size == 1
portfolio = int(portfolio.item())

# Tell GPT to recommend portfolio
session1.inject(line=f"Then, I would recommend portfolio {portfolio}.",role= "assistant")
session1(1)


Customer: Hi. I am not sure about how to invest. Could you help me?
Advisor: Of course, I'd be happy to help. Before we get started, can you tell me a little bit about your financial goals? Are you looking to save for retirement, a down payment on a house, or something else?

Customer: I am planning for retirement and also want to make sure I have a good emergency fund in case anything unexpected happens.


 message: no, the customer does not mention their risk appetite in this dialogue. 
 sentiment:   Neutral
Advisor: Great, those are both important goals. Before we discuss any specific investment options, I'd like to ask a few questions to better understand your financial situation. Would you mind sharing your age and income range with me? This will help me recommend investments that are appropriate for your financial goals and risk tolerance.

Customer: Sure, I am 50 years old with an annual income of 120k.


 message: no, the customer did not mention his risk appetite in the given 

KeyboardInterrupt: 

# GPT/Human

In [2]:
session1 = ChatSession(gpt_name='Advisor')

user_input = input("> ")
print('Customer: ', user_input)

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

investor_profile = {i:False for i in ['age','income','risk appetite']}
while True:
    session1.chat(user_input=user_input)
    user_input = input("> ")
    print('Customer: ', user_input)
    update_investor_profile(investor_profile=investor_profile,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'Customer: {user_input}')
    if not len([i for i in investor_profile.values() if not i]):
        break

# Rule based portfolio by using ``investor_profile``
portfolio = RuleBasedPortfolios.where(lambda x: x['age'].apply(lambda y: y in investor_profile['age'].lower())*\
                            x['income'].apply(lambda y: y in investor_profile['income'].lower())*\
                                x['risk appetite'].apply(lambda y: y in investor_profile['risk appetite'].lower()))['portfolio'].dropna().values
assert portfolio.size == 1
portfolio = int(portfolio.item())

# Tell GPT to recommend portfolio
session1.inject(line=f"Then, I would recommend portfolio {portfolio}.",role= "assistant")
session1(1)

Customer:  
Advisor: Hello, welcome to our bank. How can I assist you in terms of financial advice?

Customer:  i wanna invest
Advisor: Great! We have a wide range of investment options available. Could you tell me a bit more about what you are looking for in an investment? Are you looking for long-term or short-term opportunities? And what are your financial goals?

Customer:  i think long term
Advisor: Thank you for letting me know. Before we proceed, I would like to get to know you better so that I can suggest an investment portfolio that fits your risk profile and financial goals. May I ask, how old are you?

Customer:  i m in my 20s
Advisor: Thank you. Could you also share with me your approximate yearly income? This will help me recommend an investment portfolio that fits your financial situation.

Customer:  about 80k
Advisor: Thank you for sharing that information. Lastly, could you tell me your attitude towards investment risk? Are you comfortable with assuming high risks for 